In [0]:
INBOUND_PATH = "/mnt/yahoo-finances-bg/inbound/"

FILE_TEST_PATH = "/FileStore/tables/finances_AAPL_21*.csv"

RAW_PATH = "/mnt/yahoo-finances-bg/raw/"

PROCESSING_PATH = "/mnt/yahoo-finances-bg/processing/"

CURATED_PATH = "/mnt/yahoo-finances-bg/curated/"


In [0]:
get_sch_data = spark.read.format('csv').option('header', 'true').load(FILE_TEST_PATH)
data_static_sch = get_sch_data.schema

get_sch_data.createOrReplaceTempView("vw_data")

In [0]:
df_silver = spark.readStream.format("delta").load(PROCESSING_PATH)

df_silver.isStreaming

Out[3]: True

In [0]:
from pyspark.sql.functions import col

transf_df_silver = df_silver.withColumn("amplitude", col("high")+col("low"))

In [0]:
display(transf_df_silver)

open,close,high,volume,low,timestamp,day,year,month,ticker,amplitude
16.125,6417722,16.3799991607666,16.229999542236328,16.3700008392334,1642689000,21,2022,1,VALE,32.75
16.21500015258789,3358880,16.309999465942383,16.250699996948242,16.229999542236328,1642692600,21,2022,1,VALE,32.53999900817871
16.235000610351562,3779589,16.3700008392334,16.3700008392334,16.2549991607666,1642696200,21,2022,1,VALE,32.625
16.300100326538086,2881656,16.399999618530273,16.334999084472656,16.364999771118164,1642699800,21,2022,1,VALE,32.76499938964844
16.280000686645508,4782015,16.3799991607666,16.283599853515625,16.334999084472656,1642703400,21,2022,1,VALE,32.71499824523926
16.020000457763672,6965979,16.28499984741211,16.02669906616211,16.28499984741211,1642707000,21,2022,1,VALE,32.56999969482422
15.9399995803833,6554616,16.04990005493164,15.989999771118164,16.030000686645508,1642710600,21,2022,1,VALE,32.07990074157715
166.97999572753906,169.22999572753906,169.33999633789062,19181838,166.75999450683594,1642689000,21,2022,1,AAPL,336.09999084472656
169.23989868164062,169.35000610351562,169.63999938964844,9252283,168.94000244140625,1642692600,21,2022,1,AAPL,338.5800018310547
169.36000061035156,169.10000610351562,169.47999572753906,8884453,168.77999877929688,1642696200,21,2022,1,AAPL,338.25999450683594


In [0]:
transf_df_silver \
 .writeStream \
 .format("delta") \
 .partitionBy("year", "month", "ticker") \
 .option("checkpointLocation", CURATED_PATH + "_checkpoint/") \
 .start(CURATED_PATH)

Out[9]: <pyspark.sql.streaming.StreamingQuery at 0x7f567c173490>

In [0]:
%sql 

SELECT * FROM delta.`/mnt/yahoo-finances-bg/curated`

open,close,high,volume,low,timestamp,day,year,month,ticker,amplitude
166.97999572753906,169.22999572753906,169.33999633789062,19181838,166.75999450683594,1642689000,21,2022,1,AAPL,336.09999084472656
169.23989868164062,169.35000610351562,169.63999938964844,9252283,168.94000244140625,1642692600,21,2022,1,AAPL,338.5800018310547
169.36000061035156,169.10000610351562,169.47999572753906,8884453,168.77999877929688,1642696200,21,2022,1,AAPL,338.25999450683594
169.0749969482422,167.60000610351562,169.22000122070312,7758353,167.60000610351562,1642699800,21,2022,1,AAPL,336.82000732421875
167.61000061035156,166.64010620117188,167.91000366210938,9169710,166.4600067138672,1642703400,21,2022,1,AAPL,334.37001037597656
166.64739990234375,165.84500122070312,166.97000122070312,11023737,165.64999389648438,1642707000,21,2022,1,AAPL,332.6199951171875
165.83999633789062,164.52000427246094,165.97000122070312,13853054,164.17999267578125,1642710600,21,2022,1,AAPL,330.1499938964844
16.125,6417722,16.3799991607666,16.229999542236328,16.3700008392334,1642689000,21,2022,1,VALE,32.75
16.21500015258789,3358880,16.309999465942383,16.250699996948242,16.229999542236328,1642692600,21,2022,1,VALE,32.53999900817871
16.235000610351562,3779589,16.3700008392334,16.3700008392334,16.2549991607666,1642696200,21,2022,1,VALE,32.625
